# Compute Graph Replacement Score & Visualize

### Config

In [1]:
huggingface_path = "georglange/crosslayer-transcoder-topk-16k"

### Load model from HuggingFace 

In [2]:
from crosslayer_transcoder.model.serializable_module import SerializableModule


clt_model = SerializableModule.from_pretrained(huggingface_path)
assert clt_model._is_folded 

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from crosslayer_transcoder.model.clt import CrossLayerTranscoder
assert isinstance(clt_model, CrossLayerTranscoder)

In [4]:
from crosslayer_transcoder.utils.model_converters.circuit_tracer import (
    CircuitTracerConverter,
)
save_dir = "circuit_tracing_replacement_score"
feature_input_hook = "hook_resid_mid"
feature_output_hook = "hook_mlp_out"

converter = CircuitTracerConverter(
save_dir=save_dir,
feature_input_hook=feature_input_hook,
    feature_output_hook=feature_output_hook,
)
converter.convert_and_save(clt_model) 

Converting CLT encoder: 100%|██████████| 12/12 [00:33<00:00,  2.79s/it]


### Load model from local converted checkpoint

In the future, this could be loaded from huggingface using the `ReplacementModel.from_pretrained`

In [5]:
from circuit_tracer.transcoder.cross_layer_transcoder import load_clt
scan_id = "dallas-austin"
circuit_tracer_clt = load_clt(
    clt_path=save_dir,
    lazy_decoder=False,
    lazy_encoder=False,
    feature_input_hook=feature_input_hook,
    feature_output_hook=feature_output_hook,
    scan=scan_id
)


In [6]:
from circuit_tracer import ReplacementModel

rm = ReplacementModel.from_pretrained_and_transcoders(
    "gpt2",
    circuit_tracer_clt,
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loaded pretrained model gpt2 into HookedTransformer


### Attribution

In [7]:
prompt = (
    "Mary and John went to the store, Jon gave a drink to"  # What you want to get the graph for
)
max_n_logits = 10  # How many logits to attribute from, max. We attribute to min(max_n_logits, n_logits_to_reach_desired_log_prob); see below for the latter
desired_logit_prob = 0.95  # Attribution will attribute from the minimum number of logits needed to reach this probability mass (or max_n_logits, whichever is lower)
max_feature_nodes = 8192  # Only attribute from this number of feature nodes, max. Lower is faster, but you will lose more of the graph. None means no limit.
batch_size = 256  # Batch size when attributing
verbose = True  # Whether to display a tqdm progress bar and timing report


In [8]:
from pathlib import Path
import torch

from circuit_tracer import attribute
from circuit_tracer.utils import create_graph_files

torch.cuda.empty_cache()

graph = attribute(
    prompt=prompt,
    model=rm,
    max_n_logits=max_n_logits,
    desired_logit_prob=desired_logit_prob,
    batch_size=batch_size,
    max_feature_nodes=max_feature_nodes,
    offload=None,
    verbose=verbose
)

Phase 0: Precomputing activations and vectors
Precomputation completed in 0.38s
Found 2496 active features
Phase 1: Running forward pass
Forward pass completed in 0.03s
Phase 2: Building input vectors
Selected 10 logits with cumulative probability 0.7510
Will include 2496 of 2496 feature nodes
Input vectors built in 0.04s
Phase 3: Computing logit attributions
sys:1: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
Logit attributions completed in 0.14s
Phase 4: Computing feature attributions
Feature influence computation: 100%|██████████| 2496/2496 [00:01<00:00, 2007.91it/s]
Feature attributions completed in 1.32s
Attribution completed in 2.25s


### Replacement Score

Now that we have a graph created for our prompt above, we can generate metrics that tell us how much of the Replacement Model's computation is attributable to the feature vs error nodes. 

[Full explanation](https://transformer-circuits.pub/2025/attribution-graphs/methods.html#evaluating-graphs-comparing)


In [9]:
from circuit_tracer.graph import compute_graph_scores
print("replacement score, completeness score")
compute_graph_scores(graph)

replacement score, completeness score


(0.7058860659599304, 0.9123725891113281)

### [Optional] Visualize the graph

In [10]:
graph_dir = 'graphs'
graph_name = 'example_graph.pt'
graph_dir = Path(graph_dir)
graph_dir.mkdir(exist_ok=True)
graph_path = graph_dir / graph_name

graph.to_pt(graph_path)

In [11]:
slug = "mary-john-store"  # this is the name that you assign to the graph
graph_file_dir = "./graph_files"  # where to write the graph files. no need to make this one; create_graph_files does that for you
node_threshold = (
    0.8  # keep only the minimum # of nodes whose cumulative influence is >= 0.8
)
edge_threshold = (
    0.98  # keep only the minimum # of edges whose cumulative influence is >= 0.98
)

create_graph_files(
    graph_or_path=graph,  # the graph to create files for
    slug=slug,
    output_path=graph_file_dir,
    node_threshold=node_threshold,
    edge_threshold=edge_threshold,
)


In [12]:
from circuit_tracer.frontend.local_server import serve
from IPython.display import IFrame

port = 8048
server = serve(data_dir="./graph_files/", port=port)


print(
    f"Open your graph here: f'http://localhost:{port}/index.html'"
)


Open your graph here: f'http://localhost:8048/index.html'
